In [36]:
import csv
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

path = "C://Users//HP//OneDrive//Desktop//my_new//chromedriver-win64//chromedriver.exe"

# Set up the Chrome driver
options = Options()
options.add_argument("--start-maximized")
service = Service(path)
browser = webdriver.Chrome(service=service, options=options)

# List of categories to search for
categories = [
    "Smartphones", "Laptops", "Kitchen", "Shoes", "Computers", 
    "Electronics", "Clothing", "Beauty", "Books", "Toys & Games"
]

# Function to search for products in a category with discount above 50%
def search_products_in_category(category):
    print(f"Searching for best sellers in {category} with above 50% discount...")
    browser.get("https://www.amazon.in")  # Go to Amazon homepage
    
    input_search = browser.find_element(By.ID, "twotabsearchtextbox")
    search_btn = browser.find_element(By.ID, "nav-search-submit-button")
    
    input_search.clear()
    input_search.send_keys(f"{category} above 50% discount")
    sleep(1)
    
    search_btn.click()
    sleep(5)

def scrape_products(category):
    products = []
    for i in range(10):
        print(f"Scraping page {i + 1} for category: {category}")
        product_names = browser.find_elements(By.XPATH, "//*[contains(@class, 'a-size-medium')]")
        product_prices = browser.find_elements(By.XPATH, "//*[@class='a-price-whole']")
        ratings = browser.find_elements(By.XPATH, "//i[contains(@class, 'a-icon a-icon-star-small')]")
        discounts = browser.find_elements(By.XPATH, "//span[contains(text(), '% off')]")
        past_details = browser.find_elements(By.XPATH, "//span[@class='a-size-base a-color-secondary']")
        images = browser.find_elements(By.XPATH, "//img[contains(@class, 's-image')]")
        anchor_tags = browser.find_elements(By.XPATH, "//a[(contains(@class, 'a-link-normal') and contains(@class, 's-line-clamp-2') and contains(@class, 's-link-style') and contains(@class, 'a-text-normal')) or (contains(@class, 'a-link-normal') and contains(@class, 's-line-clamp-4') and contains(@class, 's-link-style') and contains(@class, 'a-text-normal'))]")
        
        for product, price, rating, discount, past_detail, img, anchor_tag in zip(
            product_names, product_prices, ratings, discounts, past_details, images, anchor_tags
        ):
            # Open the product in a new tab
            browser.execute_script("window.open(arguments[0]);", anchor_tag.get_attribute('href'))
            browser.switch_to.window(browser.window_handles[-1])

            try:
                shipped_from = browser.find_element(By.XPATH, "//div[@class='tabular-buybox-text a-spacing-none']//span").text
            except Exception as e:
                print(f"Error extracting 'Shipped From': {e}")
                shipped_from = "N/A"
            
            try:
                sold_by = browser.find_element(By.XPATH, "//a[@id='sellerProfileTriggerId']").text
            except Exception as e:
                print(f"Error extracting 'Sold By': {e}")
                sold_by = "N/A"
            
            browser.close()
            browser.switch_to.window(browser.window_handles[0])
            
            # Append product details
            products.append([
                category,  # Add category name
                product.text,
                price.text,
                rating.get_attribute('innerText') if rating else "N/A",
                discount.text if discount else "N/A",
                past_detail.text if past_detail else "N/A",
                img.get_attribute('src') if img else "N/A",
                shipped_from,
                sold_by
            ])
            browser.switch_to.window(browser.window_handles[0])
        try:
            nxt_btn = browser.find_element(By.XPATH, "//a[@class='s-pagination-item s-pagination-next']")
            nxt_btn.click()
            sleep(7)  # Wait for the next page to load
        except Exception as e:
            print(f"No more pages to scrape")
            break
    
    return products

def scrape_all_categories():
    all_products = []
    
    for category in categories:
        search_products_in_category(category)
        category_products = scrape_products(category)
        all_products.extend(category_products)
    
    return all_products

def save_to_csv(products):
    header = ['Category', 'Product Name', 'Product Price', 'Rating', 'Discount', 'Past Details', 'Image URL', 'Shipped From', 'Sold By']
    
    with open('amazon_best_sellers.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for product in products:
            writer.writerow(product)
    
    print("Data saved to amazon_best_sellers.csv")


all_products = scrape_all_categories()

save_to_csv(all_products)

browser.quit()

Searching for best sellers in Smartphones with above 50% discount...
Scraping page 1 for category: Smartphones
No more pages to scrape
Searching for best sellers in Laptops with above 50% discount...
Scraping page 1 for category: Laptops
No more pages to scrape
Searching for best sellers in Kitchen with above 50% discount...
Scraping page 1 for category: Kitchen
No more pages to scrape
Searching for best sellers in Shoes with above 50% discount...
Scraping page 1 for category: Shoes
No more pages to scrape
Searching for best sellers in Computers with above 50% discount...
Scraping page 1 for category: Computers
No more pages to scrape
Searching for best sellers in Electronics with above 50% discount...
Scraping page 1 for category: Electronics
Error extracting 'Shipped From': Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='tabular-buybox-text a-spacing-none']//span"}
  (Session info: chrome=131.0.6778.205); For documentation on this error